In [1]:
PLAYER_PAPER = 0
PLAYER_SCISSORS = 1
PLAYER_ROCK = 2
COMPUTER_PAPER = 3
COMPUTER_SCISSORS = 4
COMPUTER_ROCK = 5
CARD_IDX_MAX = 6
FORCE_WIN = 6
EXCHANGE = 7
FORCE_DRAW = 8
DELETE = 9
ADD = 10
IDX_MAX = 11

PAPER = 0
SCISSORS = 1
ROCK = 2

In [2]:
EXCHANGE_PARAM_SET = set()

for a in range(3):
    for b in range(3):
        for c in range(3):
            for d in range(3):
                exchange_parm = [0,0,0]
                exchange_parm[a] += 1
                exchange_parm[b] += 1
                exchange_parm[c] -= 1
                exchange_parm[d] -= 1
                exchange_parm = tuple(exchange_parm)
                EXCHANGE_PARAM_SET.add(exchange_parm)

print(len(EXCHANGE_PARAM_SET))


19


In [3]:
def ret_max(a, b):
    if a[0] > b[0]:
        return a
    else:
        return b

def mod_tuple(t, idx, val):
    new_t = list(t)
    new_t[idx] = val
    return tuple(new_t)

def roll_state(state, i):
    state_p = state[i:3] + state[0:i]
    j = i+3
    state_c = state[j:6] + state[3:j]
    return tuple(state_p + state_c + state[6:])

dfs_result_dict = {}

def dfs(state):
    # return _dfs(state)
    best_i = 0
    best_state = state
    for i in range(1,3):
        tmp_state = roll_state(state, i)
        if tmp_state > best_state:
            best_i = i
            best_state = tmp_state
    ret = _dfs(best_state)
    ret = mod_tuple(ret, 1, best_i)
    return ret
    
def _dfs(*args):
    if args in dfs_result_dict:
        return dfs_result_dict[args]
    else:
        ret = __dfs(*args)
        dfs_result_dict[args] = ret
        return ret

def assert_state(state):
    assert(len(state) == IDX_MAX)
    assert(type(state) == tuple)
    if sum(state[:CARD_IDX_MAX]) == 0:
        return
    assert(sum(state[0:3])-1 == (sum(state[3:6])-state[DELETE]))

def __dfs(state):
    # print('__dfs    ', state)
    # assert(type(state) == tuple)
    assert_state(state)

    if sum(state[:CARD_IDX_MAX]) == 0:
        return (0, 0, None, None, None, None) # score, roll, magic, magicparam, card, next_state
    
    ret = (-999, None, None, None, None, None)

    # FORCE_WIN
    if state[FORCE_WIN] > 0:
        new_state = mod_tuple(state, FORCE_WIN, state[FORCE_WIN] - 1)
        tmp_ret = dfs_card(new_state, True, False)
        tmp_ret = mod_tuple(tmp_ret, 2, 'FORCE_WIN')
        ret = ret_max(ret, tmp_ret)
    
    # EXCAHNGE
    if state[EXCHANGE] > 0:
        if sum(state[:CARD_IDX_MAX]) >= 4:
            for exchange_param in EXCHANGE_PARAM_SET:
                good = True
                for ei in range(3):
                    v = exchange_param[ei]
                    if v < 0:
                        if state[ei] < -v:
                            good = False
                            break
                    if v > 0:
                        if state[ei+3] < v:
                            good = False
                            break
                if not good:
                    continue

                new_state = list(state)
                new_state[EXCHANGE] -= 1
                for i in range(3):
                    new_state[i] += exchange_param[i]
                    new_state[i+3] -= exchange_param[i]
                new_state = tuple(new_state)

                tmp_ret = dfs_card(new_state, False, False)
                tmp_ret = mod_tuple(tmp_ret, 2, 'EXCHANGE')
                tmp_ret = mod_tuple(tmp_ret, 3, exchange_param)
                ret = ret_max(ret, tmp_ret)

    # FORCE_DRAW
    if state[FORCE_DRAW] > 0:
        new_state = mod_tuple(state, FORCE_DRAW, state[FORCE_DRAW] - 1)
        tmp_ret = dfs_card(new_state, False, True)
        tmp_ret = mod_tuple(tmp_ret, 2, 'FORCE_DRAW')
        ret = ret_max(ret, tmp_ret)
    
    # DELETE
    if state[DELETE] > 0:
        for i in range(3):
            if state[i+3] > 0:
                new_state = list(state)
                new_state[DELETE] -= 1
                new_state[i+3] -= 1
                new_state = tuple(new_state)
                tmp_ret = dfs_card(new_state, False, False)
                tmp_ret = mod_tuple(tmp_ret, 2, 'DELETE')
                tmp_ret = mod_tuple(tmp_ret, 3, i)
                ret = ret_max(ret, tmp_ret)

    # ADD
    if state[ADD] > 0:
        for i in range(3):
            new_state = list(state)
            new_state[ADD] -= 1
            new_state[i] += 1
            new_state[i+3] += 1
            new_state = tuple(new_state)
            tmp_ret = dfs_card(new_state, False, False)
            tmp_ret = mod_tuple(tmp_ret, 2, 'ADD')
            tmp_ret = mod_tuple(tmp_ret, 3, i)
            ret = ret_max(ret, tmp_ret)

    # no magic
    tmp_ret = dfs_card(state, False, False)
    ret = ret_max(ret, tmp_ret)

    assert(ret[0] != -999)

    return ret

dfs_card_result_dict = {}

def dfs_card(*args):
    if args in dfs_card_result_dict:
        return dfs_card_result_dict[args]
    else:
        ret = _dfs_card(*args)
        dfs_card_result_dict[args] = ret
        return ret

def _dfs_card(state, force_win, force_draw):
    # print('_dfs_card', state, force_win, force_draw)
    assert_state(state)
    # assert(type(state) == tuple)
    ret = (-999, None, None, None, None, None)
    for i in range(3):
        if state[i] <= 0:
            continue
        new_state = list(state)
        new_state[i] -= 1
        if sum(state[3:6]) == 0:
            tmp_ret = dfs(tuple(new_state))
            if tmp_ret[0] > ret[0]:
                ret = (tmp_ret[0], 0, None, None, i, tuple(new_state))
            continue
        j = i
        if not force_draw:
            j += 1
        j = j % 3
        while True:
            if state[j+3] > 0:
                break
            j += 2
            j = j % 3
        # new_state = list(state)
        # new_state[i] -= 1
        new_state[j+3] -= 1
        new_state = tuple(new_state)
        tmp_ret = dfs(new_state)
        tmp_ret = list(tmp_ret)
        if force_win or ((i-j+3)%3 == 1):
            tmp_ret[0] += 1
        if tmp_ret[0] > ret[0]:
            ret = (tmp_ret[0], 0, None, None, i, new_state)
    assert(ret[0] != -999)
    # if (ret[0] == -999):
    #     print(state)
    #     assert(False)
    return ret


In [4]:
init_state = (3,3,3,3,3,3,1,1,1,1,1)

# print(dfs(init_state))
dfs(init_state)
print('DONE')


DONE


In [5]:
state = init_state

roll = 0

while True:
    # print(state)
    state_roll = roll_state(state, roll)
    print(state_roll)
    ret = dfs(state)
    if ret[5] is None:
        break
    roll += (3-ret[1])
    roll %= 3
    ret_roll = list(ret)
    ret_roll[1] = 0
    if ret_roll[3] is None:
        pass
    elif type(ret_roll[3]) == int:
        ret_roll[3] = (ret_roll[3] - roll) % 3
    elif type(ret_roll[3]) == tuple:
        rr3 = ret_roll[3]
        ret_roll[3] = rr3[roll:3] + rr3[0:roll]
    else:
        assert(False)
    ret_roll[4] = (ret_roll[4] - roll) % 3
    ret_roll[5] = roll_state(ret_roll[5], roll)
    print(ret_roll)
    state = ret[5]
    if state == None:
        break


(3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1)
[6, 0, None, None, 0, (2, 3, 3, 3, 2, 3, 1, 1, 1, 1, 1)]
(2, 3, 3, 3, 2, 3, 1, 1, 1, 1, 1)
[6, 0, None, None, 1, (2, 2, 3, 3, 2, 2, 1, 1, 1, 1, 1)]
(2, 2, 3, 3, 2, 2, 1, 1, 1, 1, 1)
[6, 0, 'DELETE', 1, 0, (1, 2, 3, 3, 0, 2, 1, 1, 1, 0, 1)]
(1, 2, 3, 3, 0, 2, 1, 1, 1, 0, 1)
[6, 0, 'FORCE_DRAW', None, 1, (1, 1, 3, 2, 0, 2, 1, 1, 0, 0, 1)]
(1, 1, 3, 2, 0, 2, 1, 1, 0, 0, 1)
[5, 0, 'EXCHANGE', (2, 0, -2), 0, (2, 1, 1, 0, 0, 3, 1, 0, 0, 0, 1)]
(2, 1, 1, 0, 0, 3, 1, 0, 0, 0, 1)
[4, 0, None, None, 0, (1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 1)]
(1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 1)
[3, 0, None, None, 0, (0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1)]
(0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1)
[2, 0, 'FORCE_WIN', None, 1, (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1)]
(0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1)
[1, 0, 'ADD', 1, 2, (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0)]
(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0, 0, None, None, 1, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
